<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/LSTM__wrheadx3_firstpartialsuccessameba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [ ]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):

    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}

def plot_all_model( sample = None, label=None,fig=None,ax=None):
    X = label[:,0]
    Y = label[:,1]
    img = sample[:,:].detach().squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    predres = 1000

    s = [.001 for x in range(predres)]
    
    assert len(s) == predres
    c = ['red' for x in range(predres)]
    assert len(c) == predres
    ax.scatter(X.detach().cpu().numpy(),Y.detach().cpu().numpy(),s = s,c = c)


def getXYs(model=None,loader_disp = None):

    outputs = torch.zeros(100,1000,2).cuda()
    h0 = None
    c0 = None
    
    for samples,labels in loader_disp:
      for i in range(38):
        optimizer.zero_grad()
        if i == 0:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i)
          outputs[:,i*25:(i+1)*25,:] = out.detach()
        else:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i,o = o,output=output)
          outputs[:,i*25:(i+1)*25,:] = out.detach()

      return samples, outputs


          
        

def plot_all( sample = None, labels = None,fig = None,ax = None):
    img = sample[:,:].squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    X = labels[:,0]
    Y = labels[:,1]
    s = [.001 for x in range(numpoints)]
    c = ['red' for x in range(numpoints)]
    ax.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
    
class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,ddataset, model):
        #model.setBatchSize(batch_size = 1)
        fig = plt.figure()
          
        if model != None:
          model = model.eval()
          loader_disp = data.DataLoader(
            ddataset, 
            batch_size=100,
            num_workers=2)
          samples, outputs = getXYs(model=model,loader_disp = loader_disp)
          for i in range(100):
            ax = fig.add_subplot(10,10,i+1)
            plot_all_model(samples[i,:,:],outputs[i,:,:],fig=fig,ax=ax)
            plt.axis('off')

        else:
          for i in range(100):
            sample, labels = dataset[i]
            ax = fig.add_subplot(10,10,i+1)
            plot_all(sample = sample,labels = labels,fig = fig, ax= ax)
            plt.axis('off')

        fig.savefig(title,dpi=450)
        plt.close(fig) #where f is the figure
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)


In [ ]:
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [ ]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
dataset = DonutDataset(length = 100*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([2000, 1]) torch.Size([2000, 1]) torch.Size([2000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [ ]:
mini_batch = 100
test_dataset = DonutDataset(length = 1000)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([1000, 1]) torch.Size([1000, 1]) torch.Size([1000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [ ]:
pred_dataset = DonutDataset(length = 100)

x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [ ]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 256
        self.num_layers = 2
        self.seq_len = 25
        self.full_len = 1000
        self.longmem_len = int(self.full_len/self.seq_len)
        self.longtermMem = None
        self.longtermMemP = None
        self.lstm_cell = nn.LSTM(self.hidden_size*2, self.hidden_size,self.num_layers)

        self.inject_image =  nn.Sequential(
            nn.Linear(2*1000+1024+6*self.hidden_size,self.hidden_size*2),
            nn.ReLU()
        )

        
        self.fc_out = nn.Sequential(
            nn.Linear(2+self.hidden_size*8,10000),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(10000,2),
            nn.Sigmoid()
        )
        self.hi_1 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )
        self.hi_2 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )

        self.ci_1 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )
        self.ci_2 = nn.Sequential(
            nn.Linear(2+1024+self.hidden_size*2,self.hidden_size),
        )

        self.alpha_1 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.seq_len),
            nn.Softmax()
        )
        
        self.alpha_long1 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.alpha_long2 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.alpha_long3 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.long_term_write = nn.Sequential(
            nn.Linear(self.hidden_size*(self.seq_len+1),self.hidden_size),
            nn.ReLU()
        )
        

        self.embs = None #= torch.empty((self.batch_size, 1000, self.hidden_size)).cuda()
        self.ltm_temp = None
        

            
        
    def forward(self, features, p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        #print('a')
        assert features != None
        self.track +=1
        batch_size = features.size(0)
        

        
        
        if o == None:
          out = p0
          self.longtermMem = torch.zeros(batch_size,self.longmem_len,self.hidden_size).cuda()
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          self.longtermMemP = torch.zeros(batch_size,1000,2).cuda()
          self.ltm_temp = torch.zeros(batch_size,self.hidden_size).cuda()
        else:

          self.ltm_temp = self.long_term_write(torch.cat([torch.flatten(self.embs,start_dim=1),output.squeeze()],dim=1))
          self.longtermMem[:, ind-1, :] = self.ltm_temp
        
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          
          out = o

        #print('c')
        if h0 == None and c0 == None:
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,torch.zeros(batch_size,self.hidden_size).cuda(),torch.zeros(batch_size,self.hidden_size).cuda()],dim=1)
          h_0_1 = self.hi_1(combin_init)  
          h_0_2 = self.hi_2(combin_init)  
          c_0_1 = self.ci_1(combin_init)  
          c_0_2 = self.ci_2(combin_init)  
          
          h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        else:
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),h0[0,:,:]],dim=1)
          h_0_1 = self.hi_1(combin_init)  
          
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),h0[1,:,:]],dim=1)
          h_0_2 = self.hi_2(combin_init)  
          
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),c0[0,:,:]],dim=1)
          c_0_1 = self.ci_1(combin_init)  
          
          combin_init = torch.cat([torch.flatten(features,start_dim=1),out,output.squeeze(),c0[1,:,:]],dim=1)
          c_0_2 = self.ci_2(combin_init)
          
          h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        
        
        hidden_state = h0
        cell_state = c0
        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        
        output = None
        #print('b')
        for t in range(self.seq_len):
        
            # for the first time step the input is the feature vector
            if output == None:
              output = torch.zeros(batch_size,self.hidden_size).cuda()
              outesread1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl2 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl3 = torch.zeros(batch_size,self.hidden_size).cuda()
            
            combin_lstm = torch.cat([torch.flatten(features,start_dim=1),torch.flatten(self.longtermMemP,start_dim=1),output.squeeze(),outesread1,outesreadl1,outesreadl2,outesreadl3,self.ltm_temp],dim=1)
            combin_lstm = self.inject_image(combin_lstm)
            
            a1 = self.alpha_1(combin_lstm)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            
            
            output, (hidden_state, cell_state) = self.lstm_cell(combin_lstm.unsqueeze(0), (hidden_state, cell_state))

            outputs[:, t, :] = out
            self.longtermMemP[:,self.seq_len*ind+t,:] = out.detach()
            
            self.embs[:, t, :] = output.squeeze().clone().detach()
            

            a1 = self.alpha_1(combin_lstm)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            
            #print('d')
            al1 = self.alpha_long1(combin_lstm)
            outesreadal1 = self.longtermMem.detach()*al1.unsqueeze(2)#clone
            outesreadl1 = torch.sum(outesreadal1,dim=1)
            
            al2 = self.alpha_long2(combin_lstm)
            outesreadal2 = self.longtermMem.detach()*al2.unsqueeze(2)#clone
            outesreadl2 = torch.sum(outesreadal2,dim=1)
            
            al3 = self.alpha_long3(combin_lstm)
            outesreadal3 = self.longtermMem.detach()*al3.unsqueeze(2)#clone
            outesreadl3 = torch.sum(outesreadal3,dim=1)
            

            combOut = torch.cat([combin_lstm, output.squeeze(),out,outesread1,outesreadl1,outesreadl2,outesreadl3,self.ltm_temp],dim=1)#,outesread],dim=1)
            

            out = self.fc_out(combOut)
            out = out*32.0
            # build the output tensor
            
        
            
        #self.embs = self.embs.detach().clone()
        return outputs, hidden_state.detach(), cell_state.detach(), out.detach(),output.detach()#, captions

In [ ]:
model = DecoderRNN().cuda()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [ ]:
for seql in range(3,23,4):
  #seql = 19
  for epoch in range (5):
    loss = None
    model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 3
train loss tensor(14.4892, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(19.4430, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(27.3720, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(5.3784, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(12.8382, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.6935, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(6.5854, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(5.5399, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.3041, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.8830, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(2.9521, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
train loss tensor(3.2381, device='cuda:0', grad_fn=<MeanBackward0>)


In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('intermediate1.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate1.png


In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001


In [ ]:
#for seql in range(15,20,4):
seql = 23
for epoch in range (20):
  model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
tensor(11.4996, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(18.2976, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(13.2144, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(13.3206, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.8572, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.8426, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.1867, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.1035, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(3.6353, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.1551, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.4415, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.4992, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(5.2440, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(4.7278, device='cuda:0',

In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('intermediate2.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate2.png


In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [ ]:

model = model.train()
#for seql in range(15,20,4):
seql = 23
for epoch in range (10):
  loss = None
  model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
tensor(1.0172, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.0402, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9933, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9338, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9865, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.0132, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9311, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9356, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9979, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9136, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9610, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.0273, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9865, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(0.9679, device='cuda:0', gra

KeyboardInterrupt: ignored

ran so well, interrupted

In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('intermediate3.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate3.png


In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.0001



In [ ]:
for seql in range(23,39,4):
  #seql = 19
  for epoch in range (5):
    loss = None
    model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
train loss tensor(0.7094, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(9.6061, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(13.3134, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(5.2901, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(8.4591, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(4.4917, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(5.5698, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(3.6414, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(3.6307, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(6.2501, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(2.3150, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
train loss tensor(2.0176, device='cuda:0', grad_fn=<MeanBac

In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('intermediate4.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate4.png


In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.0001



In [ ]:
#for seql in range(15,20,4):
seql = 38
for epoch in range (20):
  model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 38
tensor(9.6799, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(13.3224, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(20.8074, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(18.7024, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(10.8698, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(20.0969, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(20.5632, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(31.8051, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(22.7785, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(35.1576, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.5260, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(3.9495, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(7.1489, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(23.3061, device='cu

In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('intermediate5.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  intermediate5.png


In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [ ]:

model = model.train()
#for seql in range(15,20,4):
seql = 38
for epoch in range (60):
  loss = None
  model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 38
tensor(1.6512, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.4455, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.5310, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.5581, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.5128, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.6047, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.6322, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.5553, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.6725, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.3991, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.6423, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.5041, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.4427, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 38
tensor(1.5869, device='cuda:0', gra

In [ ]:
writer.flush()
writer.close()


In [ ]:
#%load_ext tensorboard

In [ ]:
#!kill 3285
#%tensorboard --logdir runs

In [ ]:
#!nvidia-smi

In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  lstmpredictions.png


In [ ]:
for name, param in model.named_parameters():
  print(name)

In [ ]:
with torch.no_grad():
      model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        